In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
from implementations import *

## Do your thing crazy machine learning thing here :) ...

In [4]:
# Separate the data according to the value of column 24 (PRI_jet_num) 

def separate(y, tX, ids):
    
    split_x = []
    split_y = []
    split_ids = []
    
    jet_column_nbr = 22
    
    for i in range(4):
        
        split_x.append(tX[np.where(tX[:,jet_column_nbr] == i)])
        split_y.append(y[np.where(tX[:,jet_column_nbr] == i)])
        split_ids.append(ids[np.where(tX[:,jet_column_nbr] == i)])
    
    
    
    return split_x, split_y, split_ids

In [5]:
split_x, split_y, split_ids = separate(y, tX, ids)

In [6]:
#remove the columns from each set of data given a boolean array

def removeNone(data, selection):
   
    cleaned=[]
    
    for i in range(4):
        curr_data = data[i]
        
        cleaned.append(curr_data[:,selection[i]])
      
    return cleaned
    
    

In [7]:
#print statistics about the None values (-999) for each columns
#returns a boolean array that can be used to filter the columns that have 100% of undefined values (-999)
def dataStatistics(data):
    
    stats=[]
    
    for i in range(len(data)):
        
        print("Statistics ")
        print("Type :")
        print(i)
        
        
        nones = (data[i] == -999)
    
        mean = np.sum(nones, axis=0)/nones.shape[0]
        print(mean) 
        stats.append(mean != 1)
    
    return stats
    

In [8]:
selection = dataStatistics(split_x)

Statistics 
Type :
0
[0.26145747 0.         0.         0.         1.         1.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         1.
 1.         1.         1.         1.         1.         0.        ]
Statistics 
Type :
1
[0.09751883 0.         0.         0.         1.         1.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         1.         1.         0.        ]
Statistics 
Type :
2
[0.05859584 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
Statistics 
Type :
3
[0.0

In [9]:
cleaned = removeNone(split_x, selection)



# Now we can either drop the lines with residual Nones or replace the Nones by the median value

In [10]:
#replace the value of column 0 (can be None sometimes) by the median value of this column

def putMedianInsteadOfNone(cleaned):
    
    completed_data = []
    
    for i in range(len(cleaned)):
        #current PRI_jet_num
        current = cleaned[i]
        
        median = np.median(current[np.where(current[:,0] != -999)], axis = 0)
        
        #replace -999 by median value
        current[np.where(current[:,0] == -999)] = median
        
        completed_data.append(current)
    
    
    return completed_data
    
    

In [11]:
cleaned_with_median = putMedianInsteadOfNone(cleaned)



In [172]:
#Instead of putting the median we can simply drop the data where columns 0 == -999
def dropLineIfNone(cleaned, split_y, split_ids):
    
    res_x=[]
    res_y=[]
    res_ids=[]
    
    for i in range(len(cleaned)):
        
        current = cleaned[i]
        
        drop_indexes = np.where(current[:,0] != -999)
        
        res_x.append(current[drop_indexes])
        res_y.append(current[drop_indexes])
        res_ids.append(current[drop_indexes])
        
    return res_x, res_y, res_ids

In [173]:
dropped_x, dropped_y, dropped_ids = dropLineIfNone(cleaned, split_y, split_ids)


## At this point, the first values in each of the split data has a PRI_jet_num = 0, then 1 and so on. The data is clean and we can work with it.

## Features Expension

In [77]:
#method to perform polynomial feature expension

def build_poly(x, degree):
   
    x_extended = x

    for d in range (2, degree +1):
        x_extended = np.c_[x_extended, x**d]
        

    return x_extended

## Cross Validation

In [13]:
#method to split the training set into a (new) training set and a test set (same as in lab03)

def split_data(x, y, ratio, seed=1):
    """
    split the dataset based on the split ratio. If ratio is 0.8 
    you will have 80% of your data set dedicated to training 
    and the rest dedicated to testing
    """
    # set seed
    np.random.seed(seed)
 
    # split the data based on the given ratio

    training_nbr = int(x.shape[0] * ratio)
    indexes = np.random.choice(x.shape[0],training_nbr, replace=False)
    
    x_train = x[indexes]
    y_train = y[indexes]
    x_test = np.delete(x, indexes, axis = 0)
    y_test = np.delete(y, indexes, axis = 0)
    
    
    return x_train, y_train, x_test, y_test

In [117]:
#perform cross-validation 

def crossValidation(x, y, splitRatio, degrees, seed =1):
    
    x_train, y_train, x_test, y_test = split_data(x, y, splitRatio, seed)
    
    a_training = []
    a_testing = []
    weights = []
    degr = []
    
    # define parameter (just add more for loops if there are more parameters for the model)
    lambdas = np.arange(0.1,5,0.5)
    
    for ind, lambda_ in enumerate(lambdas):
        
        for ind_d, d in enumerate(degrees):
            
            
            #perform polynomial feature expension
            x_test_poly = build_poly(x_test,d)
            x_train_poly = build_poly(x_train, d)
           
            
            #normalize data (DANGER: the test set must be normalized with the training set's mean and std)
            mean = np.mean(x_train_poly, axis =0)
            std = np.std(x_train_poly, axis = 0)
            
            for i in range(len(std)):
        
                if(std[i] == 0):
                    std[i] = 1
            
            x_train_ready = (x_train_poly - mean) / std
            x_test_ready = (x_test_poly - mean) / std
            
            
            #add bias term
            bias_tr = np.ones(shape=x_train.shape)
            bias_te = np.ones(shape=x_test.shape)
            
            x_train_ready = np.c_[bias_tr, x_train_ready]
            x_test_ready = np.c_[bias_te, x_test_ready]
            
            
            #Models
        
            #ideal : lambdas = np.arange(0,0.03,0.001)
            w_star, e_tr = ridge_regression(y_train,x_train_ready, lambda_)
        
            #ideal : lambdas = np.arange(0,0.3,0.1)
            #w_star, e_tr = logistic_regression(y_train, x_train_ready,np.ones(x_train_ready.shape[1])  ,400, lambda_)
        
            #don't usel least squares with lambda bigger than 0.35 ideal: lambdas = np.arange(0.001,0.13,0.01)
            #w_star, e_tr = least_squares_GD(y_train, x_train_ready,np.ones(x_train_ready.shape[1])  ,400, lambda_)    
            #w_star, e_tr = least_squares_SGD(y_train, x_train,np.ones(x_train.shape[1])  ,400, lambda_)
        
            #DON'T REALLY NEED TO DO CROSS VALIDATION FOR THIS ONE ;) BUT PRACTICAL TO RUN IT HERE
            #w_star, e_tr = least_squares(y_train, x_train_ready)  
        
            degr.append(d)
        
            #compare the prediction with the reality
            accuracy_training = np.count_nonzero(predict_labels(w_star, x_train_ready) + y_train)/len(y_train)
            accuracy_testing = np.count_nonzero(predict_labels(w_star, x_test_ready) + y_test)/len(y_test)
        
            a_training.append(accuracy_training)
            a_testing.append(accuracy_testing)
            weights.append(w_star)
            print("lambda={l:.5f},degree={deg}, Training Accuracy={tr}, Testing Accuracy={te}".format(
                   l=lambda_, tr=a_training[ind*len(degrees)+ind_d], te=a_testing[ind*len(degrees)+ind_d], deg=d))
        
            #plt.plot(lambdas, a_training,'r--' , lambdas, a_testing, 'g--')
            #plt.show
    
    return weights[np.argmax(a_testing)], degr[np.argmax(a_testing)], a_testing[np.argmax(a_testing)]

In [101]:
#perform cross-validation 

def crossValidationForLogistic_reg(x, y, splitRatio, degrees, seed =1):
    
    x_train, y_train, x_test, y_test = split_data(x, y, splitRatio, seed)
    
    a_training = []
    a_testing = []
    weights = []
    degr = []
    
    index = 0
    
    # define parameter (just add more for loops if there are more parameters for the model)
    lambdas = np.arange(0.0001,0.3,0.1)
    gammas = np.arange(0.01,1,0.3)
    
    for ind, lambda_ in enumerate(lambdas):
        
        for ind_d, d in enumerate(degrees):
            
            for ind_g, gamma in enumerate(gammas):
            
                #perform polynomial feature expension
                x_test_poly = build_poly(x_test,d)
                x_train_poly = build_poly(x_train, d)
            
                #normalize data (DANGER: the test set must be normalized with the training set's mean and std)
                mean = np.mean(x_train_poly, axis =0)
                std = np.std(x_train_poly, axis = 0)
            
                for i in range(len(std)):
        
                    if(std[i] == 0):
                        std[i] = 1
            
                x_train_ready = (x_train_poly - mean) / std
                x_test_ready = (x_test_poly - mean) / std
                
               
                #add bias term
                
                bias_tr = np.ones(shape=x_train.shape)
                bias_te = np.ones(shape=x_test.shape)
            
                x_train_ready = np.c_[bias_tr, x_train_ready]
                x_test_ready = np.c_[bias_te, x_test_ready]
                
           

                #Model
        
                #ideal :lambdas = np.arange(0,0.3,0.01)
                #       gammas = np.arange(0,3,0.5)
                w_star, e_tr = reg_logistic_regression(y_train, x_train_ready, lambda_, np.ones(x_test_ready.shape[1]), 30, gamma)
        
           
                degr.append(d)
        
                #compare the prediction with the reality
                accuracy_training = np.count_nonzero(predict_labels(w_star, x_train_ready) + y_train)/len(y_train)
                accuracy_testing = np.count_nonzero(predict_labels(w_star, x_test_ready) + y_test)/len(y_test)
        
                a_training.append(accuracy_training)
                a_testing.append(accuracy_testing)
                weights.append(w_star)
                print("lambda={l:.5f},degree={deg}, gamma={ga:.5f}, Training Accuracy={tr}, Testing Accuracy={te}".format(
                       l=lambda_, tr=a_training[index], te=a_testing[index], deg=d, ga=gamma))
        
                #increment index
                index = index + 1
    
    return weights[np.argmax(a_testing)], degr[np.argmax(a_testing)], a_testing[np.argmax(a_testing)]

In [16]:
#Since we separated the data according to PRI_jet_num
# we have to make separate prediction and then put them together for the submission

def put_together(labels, indices):
    
    #First build first chunk
    ids_0 = np.matrix(indices[0]).T
    lab_0 = np.matrix(labels[0]).T
    
    unsorted_res = np.concatenate((ids_0, lab_0), axis=1)
    
    for i in range(1,len(labels)):
        ids = np.matrix(indices[i]).T
        lab = np.matrix(labels[i]).T
        by_jet_num = np.concatenate((ids, lab), axis=1)
        unsorted_res = np.concatenate((unsorted_res, by_jet_num), axis=0)
    
    sorted_res = unsorted_res[np.lexsort(np.fliplr(unsorted_res).T)]
    
    return sorted_res[0,:,:][:,1]

## Generate predictions and save ouput in csv format for submission:

In [17]:
DATA_TEST_PATH = '../data/test.csv' 
y_donotUse, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [118]:
#separate data with respect to column 24 and remove None
split_x_test, _, split_ids_test =  separate(y_donotUse, tX_test, ids_test)


split_x_cleaned_test = removeNone(split_x_test, dataStatistics(split_x_test))

#median instead of None
split_x_with_median = putMedianInsteadOfNone(split_x_cleaned_test)



#line dropped when None
#split_x_drop_lines, split_y_dropped_split_indexes_dropped = dropLineIfNone(split_x_cleaned_test, _, split_ids_test)

#degrees for polynomial feature expension
degrees = [1,2,3,4,5,6,7,8]

y_res = []

acc = []




for i in range(len(cleaned_with_median)):
    
    
    
    #training: chose either cross calidation or cross validation for logistic regression with regularization
    w_star, d, accuracy = crossValidation(cleaned_with_median[i], split_y[i], 0.9, degrees ,6)
    #w_star, d, accuracy = crossValidationForLogistic_reg(cleaned_with_median[i], split_y[i], 0.9, degrees ,6)
    
    
    #polynomial feature expension
    extended_and_normalized = normalize(build_poly(split_x_with_median[i], d))
    
    #adding bias term
    bias = np.ones(shape=split_x_with_median[i].shape)          
    x_test_ready = np.c_[bias, extended_and_normalized]
    
    #prediction
    y_res.append(predict_labels(w_star, x_test_ready))


    acc.append(accuracy)

print("Accuracy per jet nbr: \n")
print(acc)


Statistics 
Type :
0
[0.2605448 0.        0.        0.        1.        1.        1.
 0.        0.        0.        0.        0.        1.        0.
 0.        0.        0.        0.        0.        0.        0.
 0.        0.        1.        1.        1.        1.        1.
 1.        0.       ]
Statistics 
Type :
1
[0.09834149 0.         0.         0.         1.         1.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         1.         1.         0.        ]
Statistics 
Type :
2
[0.05881481 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
Statistics 
Type :
3
[0.06376737 0.         0.     

lambda=4.10000,degree=7, Training Accuracy=0.746188320859421, Testing Accuracy=0.749599679743795
lambda=4.10000,degree=8, Training Accuracy=0.7467777271160241, Testing Accuracy=0.7501000800640513
lambda=4.60000,degree=1, Training Accuracy=0.7443533768530154, Testing Accuracy=0.747898318654924
lambda=4.60000,degree=2, Training Accuracy=0.7437528497236463, Testing Accuracy=0.747097678142514
lambda=4.60000,degree=3, Training Accuracy=0.7437750914691785, Testing Accuracy=0.7471977582065652
lambda=4.60000,degree=4, Training Accuracy=0.7441643220159918, Testing Accuracy=0.7473979183346677
lambda=4.60000,degree=5, Training Accuracy=0.7443644977257815, Testing Accuracy=0.7479983987189752
lambda=4.60000,degree=6, Training Accuracy=0.7448315743819575, Testing Accuracy=0.7481985588470776
lambda=4.60000,degree=7, Training Accuracy=0.7454321015113267, Testing Accuracy=0.7487990392313851
lambda=4.60000,degree=8, Training Accuracy=0.7461549582411228, Testing Accuracy=0.7494995996797438
lambda=0.10000

lambda=4.60000,degree=3, Training Accuracy=0.6518505781713451, Testing Accuracy=0.6563507414571245
lambda=4.60000,degree=4, Training Accuracy=0.6581266388685896, Testing Accuracy=0.6640876853642811
lambda=4.60000,degree=5, Training Accuracy=0.659960738798378, Testing Accuracy=0.6655061250805931
lambda=4.60000,degree=6, Training Accuracy=0.6664087463640401, Testing Accuracy=0.6735009671179883
lambda=4.60000,degree=7, Training Accuracy=0.6678702947455902, Testing Accuracy=0.6744036105738234
lambda=4.60000,degree=8, Training Accuracy=0.6700339595065125, Testing Accuracy=0.678401031592521
lambda=0.10000,degree=1, Training Accuracy=0.6904347058953265, Testing Accuracy=0.6750694720127035
lambda=0.10000,degree=2, Training Accuracy=0.7447564014909244, Testing Accuracy=0.7312425565700675
lambda=0.10000,degree=3, Training Accuracy=0.7453739441123928, Testing Accuracy=0.7324335053592695
lambda=0.10000,degree=4, Training Accuracy=0.7477117840365233, Testing Accuracy=0.7362048431917427
lambda=0.100

lambda=4.60000,degree=6, Training Accuracy=0.7127985708299331, Testing Accuracy=0.6971020246129417
lambda=4.60000,degree=7, Training Accuracy=0.7117178712423634, Testing Accuracy=0.6965065502183406
lambda=4.60000,degree=8, Training Accuracy=0.7104607309058026, Testing Accuracy=0.6947201270345376
lambda=0.10000,degree=1, Training Accuracy=0.7188048328069384, Testing Accuracy=0.7383852052322959
lambda=0.10000,degree=2, Training Accuracy=0.7349977440216574, Testing Accuracy=0.7478574650428507
lambda=0.10000,degree=3, Training Accuracy=0.7368025266957438, Testing Accuracy=0.7519170049616599
lambda=0.10000,degree=4, Training Accuracy=0.7382563794054243, Testing Accuracy=0.7510148849797023
lambda=0.10000,degree=5, Training Accuracy=0.7386073093698301, Testing Accuracy=0.749210645015787
lambda=0.10000,degree=6, Training Accuracy=0.7388078407780618, Testing Accuracy=0.7514659449706811
lambda=0.10000,degree=7, Training Accuracy=0.7387577079260039, Testing Accuracy=0.7519170049616599
lambda=0.10

Accuracy per jet nbr: 

[0.8253602882305845, 0.7450676982591876, 0.7364033346566098, 0.7523680649526387]


In [119]:
OUTPUT_PATH = '../data/submission.csv'

#reassemble the data for the submission
y_pred = put_together(y_res, split_ids_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)